# EEC 289Q: Homework #3

Algorithm evaluation to solve the Traveling Salesman Problem (TSP) within a practical limit of 15 minutes!

## Imports

In [ ]:
import numpy as np
import random
import math
import time

## Function to Import Graphs

The following function takes in an input txt file that has line 1 as the number of nodes, line 2 with the headers, and the subsequent lines as the node connections and their costs.

In [12]:
def parse_graph(filename):
    with open(filename, 'r') as file:
        n = int(file.readline().strip())  # Reads the number of nodes
        file.readline()  # Skip the header line
        distance_matrix = np.full((n, n), np.inf)  # Initialize with infinities
        for line in file:
            i, j, dist = map(float, line.strip().split())
            i, j = int(i) - 1, int(j) - 1  # Adjust for zero-based indexing
            distance_matrix[i][j] = dist
            distance_matrix[j][i] = dist  # Ensure symmetry for undirected graph
        np.fill_diagonal(distance_matrix, 0)  # Zero distance to self
    return distance_matrix

## Function to Calculate Cost

In [ ]:
def calculate_cost(solution, distance_matrix):
    return sum(distance_matrix[solution[i], solution[(i + 1) % len(solution)]] for i in range(len(solution)))

## Simulated Annealing Algorithm Function

In [6]:

def simulated_annealing(distance_matrix, graph_type, max_time=900, SID='x'):
    # Checking initial time to keep within 15 minutes
    start_time = time.time()
    n = len(distance_matrix)
    # Shuffle to find initial solution
    current_solution = np.arange(n)
    np.random.shuffle(current_solution)
    # Calculate current cost
    current_cost = calculate_cost(current_solution, distance_matrix)

    # Initialize parameters for Simulated Annealing
    initial_temp = 10.0
    final_temp = 1e-3
    alpha = 0.995
    temp = initial_temp


    # Initializing cycle count
    visited_cycles = 0
    while time.time() - start_time < max_time:
        # Sorting and reversing a segment
        i, j = sorted(random.sample(range(n), 2))
        new_solution = current_solution.copy()
        new_solution[i:j + 1] = new_solution[i:j + 1][::-1]  # Reverse segment
        new_cost = calculate_cost(new_solution, distance_matrix)

        # Decision point to see if new solution is adopted to explore further
        if new_cost < current_cost or math.exp((current_cost - new_cost) / temp) > random.random():
            current_solution, current_cost = new_solution, new_cost

        visited_cycles += 1
        temp = max(temp * alpha, final_temp)  # Cool down

    # Saving the best solution to a text file
    solution_filename = f"solution_{SID}_{graph_type}.txt"
    with open(solution_filename, 'w') as file:
        solution_sequence = ' '.join(map(str, current_solution + 1))  # Convert zero-based indices to one-based
        file.write(solution_sequence)

    return current_solution, current_cost, visited_cycles

## Running the Algorithm

In [7]:
filename = '1000_randomDistance.txt'  # Adjust to your file path
distance_matrix = parse_graph(filename)
solution, cost, visited_cycles = simulated_annealing(distance_matrix, 'randomDistance')


In [9]:
# Format and print the results
print(f"Best path cost: {cost:.2f}")
print(f"Visited cycles: {visited_cycles:.1e}")

Best path cost: 1036.05
Visited cycles: 3.5e+06


In [10]:
filename = '1000_euclidianDistance.txt'  # Adjust to your file path
distance_matrix = parse_graph(filename)
solution, cost, visited_cycles = simulated_annealing(distance_matrix, 'euclidianDistance')

In [11]:
# Format and print the results
print(f"Best path cost: {cost:.2f}")
print(f"Visited cycles: {visited_cycles:.1e}")

Best path cost: 2747.96
Visited cycles: 3.5e+06


## Verification of Solutions

In [13]:
import numpy as np

def parse_solution_file(filename):
    with open(filename, 'r') as file:
        # Read the entire file, assuming it's a single line of space-separated indices
        solution_str = file.readline()
        # Convert the string of indices into a list of integers
        solution = np.array(solution_str.strip().split(), dtype=int)
    return solution

def verify_solution_uniqueness(solution):
    unique_solution = np.unique(solution)
    if len(unique_solution) != len(solution):
        print("The solution contains duplicate nodes.")
    else:
        print("All nodes in the solution are unique.")

    return unique_solution


In [16]:
# Example check for Random Distance Solution
filename = "solution_913082241_randomDistance.txt"  # Replace with your actual file path
solution = parse_solution_file(filename)
unique_solution = verify_solution_uniqueness(solution)

# Additional check: Verify that the solution contains all nodes expected, e.g., 1000 nodes
expected_num_nodes = 1000  # Adjust as needed
if len(unique_solution) == expected_num_nodes:
    print(f"The solution correctly contains all {expected_num_nodes} unique nodes.")
else:
    print(f"Error: The solution should contain {expected_num_nodes} unique nodes but contains {len(unique_solution)}.")

All nodes in the solution are unique.
The solution correctly contains all 1000 unique nodes.
